# Using ML anonymization to defend against membership inference attacks

In this tutorial we will show how to anonymize models using the ML anonymization module. 

We will demonstrate running inference attacks both on a vanilla model, and then on an anonymized version of the model. We will run a black-box membership inference attack using ART's inference module (https://github.com/Trusted-AI/adversarial-robustness-toolbox/tree/main/art/attacks/inference). 

This will be demonstarted using the Adult dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/adult). 

For simplicity, we used only the numerical features in the dataset.

## Load data

In [2]:
import numpy as np

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from apt.utils.dataset_utils import get_adult_dataset_pd

# requires a folder called 'datasets' in the current directory
(x_train, y_train), (x_test, y_test) = get_adult_dataset_pd()
x_train = x_train.to_numpy()
y_train = y_train.to_numpy().astype(int)
x_test = x_test.to_numpy()
y_test = y_test.to_numpy().astype(int)

# Use only numeric features (age, education-num, capital-gain, capital-loss, hours-per-week)
x_train = x_train[:, [0, 2, 8, 9, 10]].astype(int)
x_test = x_test[:, [0, 2, 8, 9, 10]].astype(int)

# get balanced dataset
x_train = x_train[:x_test.shape[0]]
y_train = y_train[:y_test.shape[0]]

print(x_train)

/Users/abigailt/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[  39   13 2174    0   40]
 [  50   13    0    0   13]
 [  38    9    0    0   40]
 ...
 [  27   13    0    0   40]
 [  26   11    0    0   48]
 [  27    9    0    0   40]]


## Train decision tree model

In [3]:
from sklearn.tree import DecisionTreeClassifier
from art.estimators.classification.scikitlearn import ScikitlearnDecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(x_train, y_train)

art_classifier = ScikitlearnDecisionTreeClassifier(model)

print('Base model accuracy: ', model.score(x_test, y_test))

Base model accuracy:  0.8087341072415699


/Users/abigailt/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


## Attack
The black-box attack basically trains an additional classifier (called the attack model) to predict the membership status of a sample.
#### Train attack model

In [4]:
from art.attacks.inference.membership_inference import MembershipInferenceBlackBox

# attack_model_type can be nn (neural network), rf (randon forest) or gb (gradient boosting)
bb_attack = MembershipInferenceBlackBox(art_classifier, attack_model_type='rf')

# use half of each dataset for training the attack
attack_train_ratio = 0.5
attack_train_size = int(len(x_train) * attack_train_ratio)
attack_test_size = int(len(x_test) * attack_train_ratio)

# train attack model
bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
              x_test[:attack_test_size], y_test[:attack_test_size])

#### Infer sensitive feature and check accuracy

In [5]:
# get inferred values for remaining half
inferred_train_bb = bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
inferred_test_bb = bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])
# check accuracy
train_acc = np.sum(inferred_train_bb) / len(inferred_train_bb)
test_acc = 1 - (np.sum(inferred_test_bb) / len(inferred_test_bb))
acc = (train_acc * len(inferred_train_bb) + test_acc * len(inferred_test_bb)) / (len(inferred_train_bb) + len(inferred_test_bb))
print(acc)

0.5434836015231544


This means that for 54% of the data, membership is inferred correctly using this attack.

# Anonymized data
## k=100

Now we will apply the same attacks on an anonymized version of the same dataset (k=100). The data is anonymized on the quasi-identifiers: age, education-num, capital-gain, hours-per-week.

k=100 means that each record in the anonymized dataset is identical to 99 others on the quasi-identifier values (i.e., when looking only at those features, the records are indistinguishable).

In [6]:
from apt.utils.datasets import ArrayDataset
from apt.anonymization import Anonymize

x_train_predictions = np.array([np.argmax(arr) for arr in art_classifier.predict(x_train)])

# QI = (age, education-num, capital-gain, hours-per-week)
QI = [0, 1, 2, 4]
anonymizer = Anonymize(100, QI)
anon = anonymizer.anonymize(ArrayDataset(x_train, x_train_predictions))
print(anon)

[[38 13  0  0 40]
 [46 13  0  0 35]
 [28  9  0  0 40]
 ...
 [26 13  0  0 40]
 [27 10  0  0 50]
 [28  9  0  0 40]]


In [7]:
# number of distinct rows in original data
len(np.unique(x_train, axis=0))

6739

In [8]:
# number of distinct rows in anonymized data
len(np.unique(anon, axis=0))

401

## Train decision tree model

In [9]:
anon_model = DecisionTreeClassifier()
anon_model.fit(anon, y_train)

anon_art_classifier = ScikitlearnDecisionTreeClassifier(anon_model)

print('Anonymized model accuracy: ', anon_model.score(x_test, y_test))

Anonymized model accuracy:  0.8308457711442786


/Users/abigailt/Library/Python/3.9/lib/python/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: The attribute `n_features_` is deprecated in 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


## Attack
### Black-box attack

In [10]:
anon_bb_attack = MembershipInferenceBlackBox(anon_art_classifier, attack_model_type='rf')

# train attack model
anon_bb_attack.fit(x_train[:attack_train_size], y_train[:attack_train_size],
                   x_test[:attack_test_size], y_test[:attack_test_size])

# get inferred values
anon_inferred_train_bb = anon_bb_attack.infer(x_train[attack_train_size:], y_train[attack_train_size:])
anon_inferred_test_bb = anon_bb_attack.infer(x_test[attack_test_size:], y_test[attack_test_size:])
# check accuracy
anon_train_acc = np.sum(anon_inferred_train_bb) / len(anon_inferred_train_bb)
anon_test_acc = 1 - (np.sum(anon_inferred_test_bb) / len(anon_inferred_test_bb))
anon_acc = (anon_train_acc * len(anon_inferred_train_bb) + anon_test_acc * len(anon_inferred_test_bb)) / (len(anon_inferred_train_bb) + len(anon_inferred_test_bb))
print(anon_acc)

0.4944724235351923


Attack accuracy is reduced to 50% (eqiuvalent to random guessing)

In [12]:
def calc_precision_recall(predicted, actual, positive_value=1):
    score = 0  # both predicted and actual are positive
    num_positive_predicted = 0  # predicted positive
    num_positive_actual = 0  # actual positive
    for i in range(len(predicted)):
        if predicted[i] == positive_value:
            num_positive_predicted += 1
        if actual[i] == positive_value:
            num_positive_actual += 1
        if predicted[i] == actual[i]:
            if predicted[i] == positive_value:
                score += 1
    
    if num_positive_predicted == 0:
        precision = 1
    else:
        precision = score / num_positive_predicted  # the fraction of predicted “Yes” responses that are correct
    if num_positive_actual == 0:
        recall = 1
    else:
        recall = score / num_positive_actual  # the fraction of “Yes” responses that are predicted correctly

    return precision, recall

# regular
print('without anonymization:', calc_precision_recall(np.concatenate((inferred_train_bb, inferred_test_bb)), 
                            np.concatenate((np.ones(len(inferred_train_bb)), np.zeros(len(inferred_test_bb))))))
# anon
print('with anonymization:', calc_precision_recall(np.concatenate((anon_inferred_train_bb, anon_inferred_test_bb)), 
                            np.concatenate((np.ones(len(anon_inferred_train_bb)), np.zeros(len(anon_inferred_test_bb))))))

without anonymization: (0.5303914835164835, 0.7588748311018303)
with anonymization: (0.49255952380952384, 0.3659255619702739)


Precision and recall are also reduced.